In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render ul li{font-size:22pt; line-height:30px;}
div.output {font-size:22pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:22pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:22pt;padding:5px;}
table.dataframe{font-size:22px;}
</style>
"""))

# 1 tensorflow v2.xx에서 v1사용하기

In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() # tensorflow v2 비활성화하고 v1 활성화
import numpy as np
import pandas as pd

## Tensorflow
- 데이터 흐름 그래프(tensor 객체의 흐름)을 사용하는 수치 계산 라이브러리
- 그래프는 node(연산)와 edge로 구성]
- sess = tf.Session()을 이용해서 실행 환경
- sess.run()을 통해서 실행결과를 확인

In [8]:
# 1단계 : tensor(상수) 정의
node1 = tf.constant('Hello, Tensorflow')
# 2단계 : 세션(연산을 실행하는 환경) 생성
sess = tf.Session()
# 3단계 : 실제 실행 및 출력
print(sess.run(node1))
print(sess.run(node1).decode())

b'Hello, Tensorflow'
Hello, Tensorflow


In [13]:
# 간단한 연산 tensor 그래프
# 1. 그래프 정의
node1 = tf.constant(10, dtype=tf.float16)
node2 = tf.constant(20, dtype=tf.float16)
node3 = tf.add(node1,node2)
# 2. 세션 생성
sess = tf.Session()
# 3. 세션 실행 & 결과
print(sess.run([node1,node2,node3]))

[10.0, 20.0, 30.0]


In [14]:
# 타입변경
import numpy as np
node1 = tf.constant(np.array([1,2,3]), dtype=tf.int16)
node2 = tf.cast(node1, dtype=tf.float32)
sess = tf.Session()
print(sess.run([node1,node2]))

[array([1, 2, 3], dtype=int16), array([1., 2., 3.], dtype=float32)]


In [16]:
# 평균값 계산 : tf.reduce_mean()
data = np.array([1.,2.,3.,4.])
m = tf.reduce_mean(data)
sess = tf.Session()
print(sess.run(m))

2.5


In [17]:
# tf.random_normal([size]) : 평균이 0이고 표준편차가 1인 난수 size개 발생 
w = tf.random_normal([1])
sess = tf.Session()
sess.run(w)

array([1.3394276], dtype=float32)

In [21]:
# 변수 노드
w = tf.Variable(tf.random_normal([1]))
sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 변수 초기화
sess.run(w)

array([-0.34450743], dtype=float32)

# 2 Tensorflow v1을 이용한 회귀분석 구현
## 2.1 독립변수 x가 1개, 타겟(종속)변수 y가 1개

In [26]:
# tensor 그래프 정의
# 데이터셋 확보
x = np.array([1,2,3])
y = np.array([2,3,4])
# weight와 bias
w = tf.Variable(tf.random.normal([1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')
# Hat, Hypothesis : 결과는 numpy 배열
H = w * x + b
# cost function (손실함수 : mse)
cost = tf.reduce_mean(tf.square(H-y))
'''
학습목적 : cost가 취소되는 w,b를 찾는것
cost함수는 2차함수이므로 곡선 그래프, 곡선위 미분값이 0이 되는 방향
(경사강하법GradientDescent)
'''
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)
# 세션 생성
sess = tf.Session()
# w, b 초기화
sess.run(tf.global_variables_initializer())
# 6000번 학습 (v2에서의 fit함수)
for step in range(6001):
    _, cost_val, w_val, b_val = sess.run([train, cost, w, b])
    if step%300==0:
        print('{}번째 cost:{}, w:{}, b:{}'.format(step, cost_val, w_val, b_val))

0번째 cost:3.173809051513672, w:[0.6951983], b:[0.04087982]
300번째 cost:0.016253231093287468, w:[1.1477137], b:[0.6642122]
600번째 cost:0.003835103241726756, w:[1.0717529], b:[0.83688897]
900번째 cost:0.0009049263899214566, w:[1.0348543], b:[0.9207678]
1200번째 cost:0.0002135240938514471, w:[1.0169307], b:[0.9615125]
1500번째 cost:5.038328163209371e-05, w:[1.0082242], b:[0.98130447]
1800번째 cost:1.1889159395650495e-05, w:[1.0039951], b:[0.9909182]
2100번째 cost:2.805662006721832e-06, w:[1.0019408], b:[0.99558824]
2400번째 cost:6.625832611462101e-07, w:[1.0009433], b:[0.99785614]
2700번째 cost:1.5672509334763163e-07, w:[1.000459], b:[0.9989573]
3000번째 cost:3.719047469985526e-08, w:[1.0002235], b:[0.999492]
3300번째 cost:8.878696000635955e-09, w:[1.0001094], b:[0.9997519]
3600번째 cost:2.1476995737401694e-09, w:[1.0000541], b:[0.99987805]
3900번째 cost:5.004778813599842e-10, w:[1.0000259], b:[0.9999411]
4200번째 cost:1.430559420922961e-10, w:[1.0000141], b:[0.9999686]
4500번째 cost:5.328478283606053e-11, w:[1.00000

In [29]:
# 최종적으로 나온 회귀식 H = W*x + b
w_,b_ = sess.run([w,b])

In [30]:
w_,b_

(array([1.0000087], dtype=float32), array([0.9999811], dtype=float32))

In [32]:
def predict(x):
    return w_[0] * x + b_[0]

In [33]:
predict(5)

6.000024616718292

## 2.2 predict을 위한 placeholder 이용
- placeholder : 외부에서 데이터를 입력받을 수 있는 노드

In [36]:
x = tf.placeholder(dtype=tf.float32)
H = 1*x + 1
sess = tf.Session()
sess.run([x,H], feed_dict={x : np.array([40,50])})

[array([40., 50.], dtype=float32), array([41., 51.], dtype=float32)]

In [37]:
# tensor 그래프 정의
# 데이터셋 확보
x_data = np.array([1,2,3])
y_data = np.array([2,3,4])
# placeholder 설정(x,y)
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
# weight와 bias
w = tf.Variable(tf.random.normal([1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')
# Hat, Hypothesis : 결과는 numpy 배열
H = w * x + b
# cost function (손실함수 : mse)
cost = tf.reduce_mean(tf.square(H-y))
'''
학습목적 : cost가 취소되는 w,b를 찾는것
cost함수는 2차함수이므로 곡선 그래프, 곡선위 미분값이 0이 되는 방향
(경사강하법GradientDescent)
'''
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)
# 세션 생성
sess = tf.Session()
# w, b 초기화
sess.run(tf.global_variables_initializer())
# 6000번 학습 (v2에서의 fit함수)
for step in range(6001):
    _, cost_val, w_val, b_val = sess.run([train, cost, w, b],feed_dict={x:x_data,
                                                                        y:y_data})
    if step%300==0:
        print('{}번째 cost:{}, w:{}, b:{}'.format(step, cost_val, w_val, b_val))

0번째 cost:1.3991860151290894, w:[1.5091416], b:[0.94853467]
300번째 cost:0.0018115121638402343, w:[1.049314], b:[0.8878976]
600번째 cost:0.00042744181700982153, w:[1.0239546], b:[0.9455456]
900번째 cost:0.00010086328256875277, w:[1.0116364], b:[0.97354805]
1200번째 cost:2.3798967959010042e-05, w:[1.0056524], b:[0.9871509]
1500번째 cost:5.61625711270608e-06, w:[1.0027457], b:[0.99375826]
1800번째 cost:1.3254681334728957e-06, w:[1.0013341], b:[0.9969676]
2100번째 cost:3.1319217441705405e-07, w:[1.0006487], b:[0.99852586]
2400번째 cost:7.427664883152829e-08, w:[1.0003158], b:[0.9992824]
2700번째 cost:1.7700680032817218e-08, w:[1.0001544], b:[0.99964994]
3000번째 cost:4.219771199132083e-09, w:[1.0000752], b:[0.99982893]
3300번째 cost:1.0179898035644896e-09, w:[1.0000367], b:[0.9999159]
3600번째 cost:2.3546911642746693e-10, w:[1.000018], b:[0.9999596]
3900번째 cost:7.409065722052688e-11, w:[1.00001], b:[0.9999779]
4200번째 cost:5.195962063386794e-11, w:[1.0000086], b:[0.9999813]
4500번째 cost:5.195962063386794e-11, w:[1.

In [39]:
# 예측하기
sess.run(H, feed_dict={x:5})

array([6.0000243], dtype=float32)

## 2.3 scale이 다른 데이터들의 회귀분석 구현(scale 조정x)

In [45]:
x_data = np.array([1, 2 , 5, 8, 10])
y_data = np.array([5, 15, 68, 80, 95])
# placeholder 설정
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
# weight & bias
w = tf.Variable(tf.random_normal([1]))
b = tf.Variable(tf.random_normal([1]))
# Hypothesis
H = w*x + b
# cost (손실함수)
cost = tf.reduce_mean(tf.square(H-y))
# 경사강하법
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train = optimizer.minimize(cost)
# Session & 변수초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 학습
for step in range(1, 6001):
    _, cost_val = sess.run([train, cost], feed_dict={x:x_data,
                                                     y:y_data})
    if step%300==0:
        print("{}/6000번째 cost:{}".format(step+1, cost_val))

301/6000번째 cost:79.19590759277344
601/6000번째 cost:79.17894744873047
901/6000번째 cost:79.16708374023438
1201/6000번째 cost:79.15877532958984
1501/6000번째 cost:79.1529541015625
1801/6000번째 cost:79.14892578125
2101/6000번째 cost:79.14607238769531
2401/6000번째 cost:79.1440658569336
2701/6000번째 cost:79.1426773071289
3001/6000번째 cost:79.14170837402344
3301/6000번째 cost:79.14105224609375
3601/6000번째 cost:79.14056396484375
3901/6000번째 cost:79.14021301269531
4201/6000번째 cost:79.1399917602539
4501/6000번째 cost:79.13981628417969
4801/6000번째 cost:79.13972473144531
5101/6000번째 cost:79.13964080810547
5401/6000번째 cost:79.13958740234375
5701/6000번째 cost:79.13954162597656
6001/6000번째 cost:79.13954162597656


## 2.4 scale이 다른 데이터들의 회귀분석 구현(scale 조정o)
### scale 조정 방법: 모든 데이터를 일정범위내로 조정
- normalization(정규화) : 모든 데이터를 0~1 사이로 조정
                      X - Xmin
   normalization = ───────────────
                     Xmax - Xmin
      * 위의 식보다 라이브러리 추천(sklearn.preprocessing.MinMaxScaler) 
- standardization(표준화) : 데이터의 평균을 0, 표준편차를 1로 조정
                        X - Xmean
   standardization = ───────────────
                      Xstd(표준편차)
      * 위의 식보다 라이브러리 추천(sklearn.preprocessing.StandardScaler)                

In [51]:
# 라이브러리 쓰지 않고 정규화
x_data = np.array([1, 2, 5, 8, 10])
y_data = np.array([5, 15, 68, 80, 95])
# x_data.min(),x_data.max(),x_data.mean(),x_data.std()
norm_scaled_x_data = (x_data - x_data.min()) / (x_data.max() - x_data.min())
norm_scaled_y_data = (y_data - y_data.min()) / (y_data.max() - y_data.min())
norm_scaled_y_data

array([0.        , 0.11111111, 0.7       , 0.83333333, 1.        ])

In [62]:
# 라이브러리를 사용하여 정규화
x_data = np.array([1, 2, 5, 8, 10]).reshape(-1,1)
y_data = np.array([5, 15, 68, 80, 95]).reshape(-1,1)
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler() # 독립변수 x를 정규화 시킬 객체
scaler_x.fit(x_data)
norm_scaled_x_data = scaler_x.transform(x_data)
scaler_y = MinMaxScaler() 
norm_scaled_y_data = scaler_y.fit_transform(y_data)
norm_scaled_y_data 

array([[0.        ],
       [0.11111111],
       [0.7       ],
       [0.83333333],
       [1.        ]])

In [65]:
# 라이브러리 쓰지 않고 표준화
x_data = np.array([1, 2, 5, 8, 10])
y_data = np.array([5, 15, 68, 80, 95])
stan_scaled_x_data = (x_data - x_data.mean()) / x_data.std()
stan_scaled_y_data = (y_data - y_data.mean()) / y_data.std()
np.column_stack([x_data, norm_scaled_x_data, stan_scaled_x_data])

array([[ 1.        ,  0.        , -1.22474487],
       [ 2.        ,  0.11111111, -0.93313895],
       [ 5.        ,  0.44444444, -0.05832118],
       [ 8.        ,  0.77777778,  0.81649658],
       [10.        ,  1.        ,  1.39970842]])

In [66]:
np.column_stack([y_data, norm_scaled_y_data, stan_scaled_y_data])

array([[ 5.        ,  0.        , -1.32373476],
       [15.        ,  0.11111111, -1.04563922],
       [68.        ,  0.7       ,  0.42826713],
       [80.        ,  0.83333333,  0.76198177],
       [95.        ,  1.        ,  1.17912508]])

In [73]:
# 라이브러리를 사용하여 표준화
x_data = np.array([1, 2, 5, 8, 10]).reshape(-1,1)
y_data = np.array([5, 15, 68, 80, 95]).reshape(-1,1)

from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
stan_scaler_x_data = scaler_x.fit_transform(x_data)
scaler_y = StandardScaler()
stan_scaler_y_data = scaler_y.fit_transform(y_data)

In [74]:
np.column_stack([y_data, norm_scaled_y_data, stan_scaled_y_data])

array([[ 5.        ,  0.        , -1.32373476],
       [15.        ,  0.11111111, -1.04563922],
       [68.        ,  0.7       ,  0.42826713],
       [80.        ,  0.83333333,  0.76198177],
       [95.        ,  1.        ,  1.17912508]])

In [77]:
# 스케일 조정된 데이터를 다시 복구 : invers_transform() 이용
scaler_y.inverse_transform(stan_scaler_y_data)

array([[ 5.],
       [15.],
       [68.],
       [80.],
       [95.]])

In [79]:
x_data = np.array([1, 2 , 5, 8, 10])
y_data = np.array([5, 15, 68, 80, 95])
# placeholder 설정
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
# weight & bias
w = tf.Variable(tf.random_normal([1]))
b = tf.Variable(tf.random_normal([1]))
# Hypothesis
H = w*x + b
# cost (손실함수)
cost = tf.reduce_mean(tf.square(H-y))
# 경사강하법
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train = optimizer.minimize(cost)
# Session & 변수초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 학습
for step in range(1, 6001):
    _, cost_val = sess.run([train, cost], feed_dict={x:norm_scaled_x_data,
                                                     y:norm_scaled_y_data})
    if step%300==0:
        print("{}/6000번째 cost:{}".format(step, cost_val))

300/6000번째 cost:0.27514755725860596
600/6000번째 cost:0.2402239590883255
900/6000번째 cost:0.21009179949760437
1200/6000번째 cost:0.18394339084625244
1500/6000번째 cost:0.16121801733970642
1800/6000번째 cost:0.14145997166633606
2100/6000번째 cost:0.12428005039691925
2400/6000번째 cost:0.10934145748615265
2700/6000번째 cost:0.0963517501950264
3000/6000번째 cost:0.08505666255950928
3300/6000번째 cost:0.07523506879806519
3600/6000번째 cost:0.06669478863477707
3900/6000번째 cost:0.05926860496401787
4200/6000번째 cost:0.05281119421124458
4500/6000번째 cost:0.04719618707895279
4800/6000번째 cost:0.04231373965740204
5100/6000번째 cost:0.03806822746992111
5400/6000번째 cost:0.034376561641693115
5700/6000번째 cost:0.03116651438176632
6000/6000번째 cost:0.028375249356031418
